# Data Wrangling the Mathematics Genealogy Project and MathSciNet data

#### Scott Atkinson

## [0. Contents](#0._Contents) <a id='0._Contents'></a>
## [ 1. MGP Dataframe](#1._MGP_Dataframe)

* [1.1. Import MGP Data](#1.1._Import_MGP_Data) Importing the raw data from `MGP_scrape.csv` into the dataframe `df`, we see we initially have 259533 rows.
* [1.2. Drops](#1.2._Drops) We then drop two columns from coming from the import, and we drop a list of rows that we have determined earlier to either be duplicates or contain no data.  We also drop the duplicates.  This leaves us with 259423 rows.
* [1.3. Cleaning `Thesis` column](#1.3._Cleaning_Thesis_column) The `Thesis` column is not likely to form a feature, the only cleaning for this column is to strip the '\n's.
* [1.4. Extracting degree title, school, and country data](#1.4._Extracting_degree_title_et_al)Next we extracted the degree title, school, and country data from the `Degree_Info` columns using string methods.  Missing country values were replaced with 'Unknown'.
* [1.5. Extracting year data](#1.5._Extracting_year_data) Then we extracted the year information.  This required some attention because some errors occurred when the year information was entered for certain records.
* [1.6. Getting MSC number](#1.6._Getting_MSC_number) Next we extracted the MSC (Math Subject Classification) number for each record.  Missing values were replaced with `np.nan`.
* [1.7. Extracting Advisor Data](#1.7._Extracting_Advisor_Data) Next we made columns for the advisor data.  The max number of advisors a record has is 6, so there are 6 pairs of advisor columns.  Each pair is made up of the advisor's name and the advisor's `MGP_ID`.  Missing values were replaced with 0 so the columns can by of type int.
* [1.8. Getting students and descendants counts](#1.8._Getting_students_and_descendants_counts) We then extracted the number of students and number of descendants for each record.  We also wrote some functions to compile the list of ancestors, students, and descendants of a given function.  The descendants function is a depth-first search.
* [1.9. Cleaning up name column](#1.9._Cleaning_up_name_column) Next we cleaned up the `Name` column.  Every name had a space at the end, and names without a middle name had a double space between first and last name
* [1.10. Extracting MathSciNet ID](#1.10._Extracting_MathSciNet_ID) We next extracted the MathSciNet IDs for each record. We then checked for duplicate MathSciNet IDs.  There were many that we sorted out in cells following the MathSciNet ID extraction.  For instance there were many MScNetIDs that were assigned to multipl MGP_IDs.  So we found the MGP_IDs which actually belonged with the MScNetIDs and appropriately replaced the mislabelled IDs.
* [1.11. More drops](#1.11._More_drops) We then dropped and rearranged several columns--mainly ones containing the raw scraped data and one created for a boolean series.



## [2. MScN data frame](#2._MScN_data_frame)

* [2.1. Import MScN data](#2.1._Import_MScN_data) Importing the raw data from `MScN_Author_scrape.csv` into the dataframe `ms`, we see we initially have 148796 rows. 
* [2.2. Drop defective rows](#2.2._Drop_defective_rows) We then dropped all duplicate rows (keeping the first of each duplicate) and dropped all entries with non-digit MScN_IDs.  We also dropped one record because it is attached to the incorrect MGP_ID.  This left us with 148190 rows
* [2.3. Align MGP and MScN MathSciNet IDs](#2.3._Align_MGP_and_MScN_MathSciNet_IDs) We next checked to see if the collection of MScNetIDs in `df` matched with those in `ms`.  Apparently the ones in `ms` are all contained in `df`, but `df` had 364 more IDs than `ms`.  It was found that these IDs are faulty and do not correspond to an actual MathSciNet Author page--possibly due to some clerical error.  Some of these could be fixed by adjusting a digit, but it was decided to just convert these 364 IDs to `None` in `df`.  At the end of this process, both dataframes have the same exact collection of functioning MathSciNet author IDs.
* [2.4. Extracting info from scraped MathSciNet data](#2.4._Extracting_info_from_scraped_MathSciNet_data)We next moved to extract the data from the raw scrape of the MathSciNet pages.
 We extracted the `Earliest_Pub`, `Total_Pubs`, `Total_Rel_Pubs`, `Total_Citations`, `CollabIDs`, `CollabNames`, `Subjects`, `Num_Collaborators`, `Num_Subjects` columns.  We replaced missing information with 0 or when the dtype was int, and with `None` otherwise.
* [2.5. Save dataframes](#2.5._Save_dataframes)Both dataframes were saved to `.csv` files: `df` to `MGP_clean.csv` and `ms` to `MScN_Author_clean.csv`.

## [3. Joining MGP and MScN](#3._Joining_MGP_and_MScN)

* [3.1. Join](#3.1._Join) We then performed `df` left join `ms` on `MScNetID` to produce the dataframe `am`. We used a left join to retain all student/advisor data to help with feature enginnering.
* [3.2. Typecast](#3.2._Typecast) We converted the `NaN`s in the numerical columns to -1s so those columns can be of type int.
* [3.3. Viable rows](#3.3._Viable_rows) There are 41037 rows in `am` with at least one students.  This will form the basis of our training/testing data for our model.

## 1. MGP Dataframe<a id='1._MGP_Dataframe'></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time


pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### 1.1. Import data<a id='1.1._Import_MGP_Data'></a>

In [2]:
df = pd.read_csv('MGP/MGP_scrape.csv', index_col = 'MGP_ID')

In [3]:
df.shape

(259553, 9)

### 1.2. Drops<a id='1.2._Drops'></a>

In [4]:
# drop unnecessary columns

df.drop(['Unnamed: 0','Unnamed: 0.1'], inplace = True, axis = 1)

In [5]:
# list of records to drop because they are true duplicates

drop_list = [243871, 
             251053, 
             219211, 
             253054, 
             261290, 
             240314, 
             248056,
             249634,
             210665,
             228171,
             234621,
             123111,
             253044,
             229095,
             206046,
             251433,
             216883,
             245041,
             257139,
             98019,
             260878,
             209542,
             221267,
             255265,
             254011,
             260874,
             231864,
             233120,
             251395,
             258014,
             251069,
             260877,
             261279,
             251816,
             242126,
             243844,
             263123,
             261189,
             260241,
             168951,
             223969,
             197269,
             173422,
             63638,
             199123,
             238993,
             238113,
             242960,
             134258,
             253326,
             207214,
             232090,
             231387,
             245248,
             215101,
             240249,
             106754,
             243236,
             233534,
             70269,
             237125,
             198385,
             241592,
             124495,
             239939,
             112397,
             233540,
             26727,
             257835,
             226455,
             244763,
             206698,
             239483,
             207383,
             239384,
             257999,
             252342,
             200574,
             206048,
             189464,
             233486,
             185355,
             246884,
             203076,
             122375,
             252389,
             245255
            ]

df.drop(drop_list, inplace=True)

In [6]:
df[df.duplicated(keep=False)].sort_values('Name').shape

(84, 7)

In [7]:
df.shape

(259466, 7)

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.shape

(259423, 7)

### 1.3. Cleaning `Thesis` column<a id='1.3._Cleaning_Thesis_column'></a>

In [10]:



# remove \n from thesis column

df['Thesis'] = df['Thesis'].str.strip('\n')

In [11]:
df.columns

Index(['Name', 'Degree_Info', 'Thesis', 'MSC', 'Advisor', 'Descendants',
       'MScNID'],
      dtype='object')

### 1.4. Extracting degree title, school, and country data from degree info column<a id='1.4._Extracting_degree_title_et_al'></a>

In [12]:
# functions to extract degree title, institutions name, and country

def extract_deg(x):
    return '%s' %(x.split('margin-right: 0.5em">')[-1].split('<span')[0])
def extract_school(x):
    return '%s' %(x.split('margin-left: 0.5em">')[-1].split('</span')[0])
def extract_country(x):
    return '%s' %(x.split('title="')[-1].split('"')[0])

In [13]:
df['Degree'] = df['Degree_Info'].apply(extract_deg)
df['School'] = df['Degree_Info'].apply(extract_school)
df['Country'] = df['Degree_Info'].apply(extract_country)

In [14]:
# cleaning up country column

df[df['Country'].str.contains('div')].shape

(14445, 10)

In [15]:
# Convert '<div style=' to 'Unknown' in 'Country' column

df['Country'] = df['Country'].apply(lambda C: 'Unknown' if C == '<div style=' else C)

In [16]:
# a function to determine the number of time </span> shows up in a value

def split_length(s):
    return len(s.split('</span>'))

### 1.5. Extracting year data<a id='1.5._Extracting_year_data'></a>

In [17]:
df[df['Degree_Info'].apply(split_length) < 3].shape

(2, 10)

2 rows have `'</span>'` show up less than twice in the `'Degree_Info'` column

In [18]:
def extract_year(x):
    if len(x.split('</span>')) > 2:
        return '%s' %(x.split('margin-left: 0.5em">')[-1].split('</span>')[1].split('</'))[0]
    else:
        return 0
df['Year'] = df['Degree_Info'].apply(extract_year)

In [19]:
# convert all year values to strings to for cleaning

df['Year'] = df['Year'].apply(str)

In [20]:
df['Year'] = df['Year'].str.strip(' ')

In [21]:
# convert all empty years to 0 so we can make this column type integers
df['Year'] = df['Year'].apply(lambda C: '0' if C == '' else C)

In [22]:
df[~df['Year'].str[:4].str.isdigit()].shape

(2, 11)

In [23]:
df.loc[73629,['Year']] = '1845'

In [24]:
df.loc[248564, ['Year']] = '2018'

In [25]:
df[~(df['Year'].str.isdigit()) & (df['Year'].str[:4].str.isdigit())].shape

(111, 11)

In [26]:
def earlier_year(s):
    if not s.isdigit() and s[:4].isdigit():
        return s[:4]
    else:
        return s
        

In [27]:
# take earlier year listed

df['Year'] = df['Year'].apply(earlier_year)

In [28]:
# based on thesis info, ID#41944 earned her PhD in 1882

df.loc[41944, ['Year']] = '1882'

In [29]:
df.loc[111235, 'Year']

'1998'

In [30]:
# some strange formatting for ID#111235, set year to 1998

df.loc[111235, 'Year'] = '1998'

In [31]:
df[(df['Year'].str.isdigit())].shape

(259423, 11)

In [32]:
df[df['Year'].str.contains('c')]

,Name,Degree_Info,Thesis,MSC,Advisor,Descendants,MScNID,Degree,School,Country,Year
MGP_ID,,,,,,,,,,,


In [33]:
# all 'Year' values are integers, convert back to integers
df['Year'] = df['Year'].apply(int)

In [34]:
# sort by year to find weird year values

df[df['Year']>0].sort_values('Year', ascending = False).head()

,Name,Degree_Info,Thesis,MSC,Advisor,Descendants,MScNID,Degree,School,Country,Year
MGP_ID,,,,,,,,,,,
235671,\nKai Kopfer,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em"">Ruprecht-Karls-Universität Heidelberg</span> 20018</span>\n<img alt=""Germany"" height=""30"" src=""img/flags/Germany.gif"" style=""border: 0; vertical-align: middle"" title=""Germany"" width=""50""/>\n</div>",A mechanochemical model for cell polarization,Mathematics Subject Classification: 92—Biology and other natural sciences,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=21566"">Willi Jäger</a><br/></p>",No students known.,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n</p>",Ph.D.,Ruprecht-Karls-Universität Heidelberg,Germany,20018
252420,\nIrene Y. Zhang,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em"">University of Washington</span> 2107</span>\n<img alt=""UnitedStates"" height=""30"" src=""img/flags/UnitedStates.gif"" style=""border: 0; vertical-align: middle"" title=""UnitedStates"" width=""57""/>\n</div>","Towards a Flexible, High-Performance Operating System for Mobile/Cloud Applications",Mathematics Subject Classification: 68—Computer science,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=69000"">Henry (Hank) M. Levy</a><br/>Advisor 2: <a href=""id.php?id=55190"">Arvind Krishnamurthy</a><br/></p>",No students known.,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n</p>",Ph.D.,University of Washington,UnitedStates,2107
238848,\nErfang Ma,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em"">University of Otago</span> 2102</span>\n<img alt=""NewZealand"" height=""30"" src=""img/flags/NewZealand.gif"" style=""border: 0; vertical-align: middle"" title=""NewZealand"" width=""60""/>\n</div>",Application of Markov Chain Monte Carlo Methods in Electrical Impedance Tomography,No MSC given,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=44383"">Colin Fox</a><br/></p>",No students known.,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n</p>",Ph.D.,University of Otago,NewZealand,2102
254349,\nTuan Anh Dao,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em"">Uppsala Universitet</span> 2024</span>\n<img alt=""Sweden"" height=""30"" src=""img/flags/Sweden.gif"" style=""border: 0; vertical-align: middle"" title=""Sweden"" width=""48""/>\n</div>",Invariant domain preserving schemes for Magnetohydrodynamics,Mathematics Subject Classification: 65—Numerical analysis,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=165208"">Murtazo Nazarov</a><br/>Advisor 2: <a href=""id.php?id=254348"">Ken Mattsson</a><br/></p>",No students known.,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n</p>",Ph.D.,Uppsala Universitet,Sweden,2024
260624,\nAnna Quaglieri,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em"">University of Melbourne</span> 2020</span>\n<img alt=""Australia"" height=""30"" src=""img/flags/Australia.gif"" style=""border: 0; vertical-align: middle"" title=""Australia"" width=""60""/>\n</div>",Using transcriptomics to study relapse in Acute Myeloid Leukaemia,Mathematics Subject Classification: 62—Statistics,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""i

IDs 235671, 252420, 238848 need correcting

In [35]:
# sort by year to find weird year values

df[df['Year']>0].sort_values('Year').head()

,Name,Degree_Info,Thesis,MSC,Advisor,Descendants,MScNID,Degree,School,Country,Year
MGP_ID,,,,,,,,,,,
259778,\nPedro Toniol Cardin,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em"">Universidade Estadual Paulista ""Júlio de Mesquita Filho""</span> 20</span>\n<img alt=""Brazil"" height=""30"" src=""img/flags/Brazil.gif"" style=""border: 0; vertical-align: middle"" title=""Brazil"" width=""43""/>\n</div>",Equações com impasse e problemas de perturbação singular,Mathematics Subject Classification: 37—Dynamical systems and ergodic theory,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=170285"">Paulo Ricardo da Silva</a><br/></p>",No students known.,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n<a href=""http://www.ams.org/mathscinet/MRAuthorID/956004"">MathSciNet</a>\n</p>",Ph.D.,"Universidade Estadual Paulista ""Júlio de Mesquita Filho""",Brazil,20
254024,\nMohammad Esmael Samei,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em""></span> 43</span>\n</div>",Fixed points of $\alpha$-$\psi$-contractive type mappings and multifunctions on intuitionistic fuzzy metric space,Mathematics Subject Classification: 46—Functional analysis,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=140610"">Sharam Rezapour</a><br/></p>",No students known.,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n</p>",Ph.D.,,Unknown,43
261324,\nJaimee Brown,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em"">Queensland University of Technology</span> 200</span>\n<img alt=""Australia"" height=""30"" src=""img/flags/Australia.gif"" style=""border: 0; vertical-align: middle"" title=""Australia"" width=""60""/>\n</div>",Secure public-key encryption from factorisation-related problems,Mathematics Subject Classification: 68—Computer science,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=261313"">Juan Gonzalez Nieto</a><br/>Advisor 2: <a href=""id.php?id=18983"">Colin Boyd</a><br/></p>",No students known.,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n</p>",Ph.D.,Queensland University of Technology,Australia,200
201288,\nGregory Chioniadis,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em""> <span style=""color:\n #006633; margin-left: 0.5em"">Ilkhans Court at Tabriz</span> 1296</span>\n<img alt=""Iran"" height=""30"" src=""img/flags/Iran.gif"" style=""border: 0; vertical-align: middle"" title=""Iran"" width=""52""/>\n</div>",,Mathematics Subject Classification: 85—Astronomy and astrophysics,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=204293"">Shams al‐Dīn al‐Bukhārī</a><br/></p>","According to our current on-line database, Gregory Chioniadis has 1 student and 169052 descendants.\n\nWe welcome any additional information.","<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n</p>",,Ilkhans Court at Tabriz,Iran,1296
184631,\nTheodore Metochites,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em""> <span style=""color:\n #006633; margin-left: 0.5em""></span> 1315</span>\n</div>",,Mathematics Subject Classification: 00—General,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=184632"">Manuel Bryennios</a><br/></p>","According to our current on-line database, Theodore Metochites has 2 students and 169050 descendants.\n\nWe welcome any additional information.","<p style="

IDs 259778, 254024, 261324 have incorrect years

In [36]:
# fixing it

df.loc[235671, 'Year'] = 2018
df.loc[252420, 'Year'] = 2017
df.loc[238848, 'Year'] = 2012
df.loc[261324, 'Year'] = 2007
df.loc[254024, 'Year'] = 2013
df.loc[259778, 'Year'] = 2011

In [37]:
# ID 254349 has a future degree year, and we are only interested in current PhDs
df.drop(254349, inplace=True)

In [38]:
#df[df['Year'] >0].sort_values('Year', ascending = False).head(30)

### 1.6. Getting MSC number<a id='1.6._Getting_MSC_number'></a>

Isolating the MSC number:

In [39]:
def extract_msc(x):
    if x != 'No MSC given':
        return '%s' %(x.split(': ')[-1].split('—')[0])
    else:
        return np.nan



In [40]:
df['msc'] = df['MSC'].apply(extract_msc)

### 1.7. Extracting Advisor Data<a id='1.7._Extracting_Advisor_Data'></a>

In [41]:
df[df['Advisor'].str.contains('Advisor 2')].shape

(41892, 12)

At least 41892 records with two advisors

In [42]:
df[df['Advisor'].str.contains('Advisor 3')].shape

(2688, 12)

At least 2688 records with three advisors

In [43]:
df[df['Advisor'].str.contains('Advisor 4')].shape

(207, 12)

At least 207 rows with four advisors

In [44]:
df[df['Advisor'].str.contains('Advisor 5')].shape

(17, 12)

At least 17 records with five advisors

In [45]:
df[df['Advisor'].str.contains('Advisor 6')].shape

(2, 12)

At least 2 records with six(!!) advisors

First getting number of advisors.

In [46]:
def num_ad(x):
    return len(x.split('id=')) - 1

# number of advisors column


df['Num_advisors'] = df['Advisor'].apply(num_ad)

Extracting Advisor Names and IDs using lambda functions.

In [47]:
# advisor names

for i in range(1,7):
    df['Advisor_%s_Name'%(i)] = df['Advisor'].apply(lambda C: C.split('id=')[i].split('">')[1].split('</a')[0] 
                                if num_ad(C)>= i
                                else None)
    
    df['Advisor_%s_MGP_ID'%(i)] = df['Advisor'].apply(lambda C: C.split('id=')[i].split('">')[0]
                                if num_ad(C) >= i
                                else '0').apply(int)

### 1.8. Getting students and descendants counts<a id='1.8._Getting_students_and_descendants_counts'></a>

In [48]:
# number of students

df['num_students'] = df.apply(lambda D: 0 if D['Descendants'] == 'No students known.'
                              else D['Descendants'].split('has ')[-1].split(' student')[0], axis = 1)



In [49]:
df['num_students'] = df['num_students'].apply(str)

In [50]:
df[~df['num_students'].str.isdigit()]

,Name,Degree_Info,Thesis,MSC,Advisor,Descendants,MScNID,Degree,School,Country,Year,msc,Num_advisors,Advisor_1_Name,Advisor_1_MGP_ID,Advisor_2_Name,Advisor_2_MGP_ID,Advisor_3_Name,Advisor_3_MGP_ID,Advisor_4_Name,Advisor_4_MGP_ID,Advisor_5_Name,Advisor_5_MGP_ID,Advisor_6_Name,Advisor_6_MGP_ID,num_students
MGP_ID,,,,,,,,,,,,,,,,,,,,,,,,,,
182831,\nWilliam Cousins,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em"">North Carolina State University</span> 2013</span>\n<img alt=""UnitedStates"" height=""30"" src=""img/flags/UnitedStates.gif"" style=""border: 0; vertical-align: middle"" title=""UnitedStates"" width=""57""/>\n</div>",Boundary Conditions and Uncertainty Quantifications for\r\nHemodynamics,No MSC given,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=45654"">Pierre-Alain Gremaud</a><br/></p>",Dissertation: \n\nBoundary Conditions and Uncertainty Quantifications for\r\nHemodynamics,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n<a href=""http://www.ams.org/mathscinet/MRAuthorID/988642"">MathSciNet</a>\n</p>",Ph.D.,North Carolina State University,UnitedStates,2013,NaN,1,Pierre-Alain Gremaud,45654,None,0,None,0,None,0,None,0,None,0,Dissertation: \n\nBoundary Conditions and Uncertainty Quantifications for\r\nHemodynamics


In [51]:
df.loc[182831,'num_students'] = '0'

In [52]:
df['num_students'] = df['num_students'].apply(int)

In [53]:
# number of descendants

df['num_descendants'] = df.apply(lambda D: 0 if D['Descendants'] == 'No students known.'
                              else D['Descendants'].split('and ')[-1].split(' desc')[0], axis = 1)

In [54]:
df['num_descendants'] = df['num_descendants'].apply(str)
df[~df['num_descendants'].str.isdigit()].shape

(63263, 27)

In [55]:
df.loc[1,'num_descendants']

'3 '

In [56]:
df['num_descendants'] = df['num_descendants'].str.strip(' ')

In [57]:
df[~df['num_descendants'].str.isdigit()]

,Name,Degree_Info,Thesis,MSC,Advisor,Descendants,MScNID,Degree,School,Country,Year,msc,Num_advisors,Advisor_1_Name,Advisor_1_MGP_ID,Advisor_2_Name,Advisor_2_MGP_ID,Advisor_3_Name,Advisor_3_MGP_ID,Advisor_4_Name,Advisor_4_MGP_ID,Advisor_5_Name,Advisor_5_MGP_ID,Advisor_6_Name,Advisor_6_MGP_ID,num_students,num_descendants
MGP_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
182831,\nWilliam Cousins,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em"">Ph.D. <span style=""color:\n #006633; margin-left: 0.5em"">North Carolina State University</span> 2013</span>\n<img alt=""UnitedStates"" height=""30"" src=""img/flags/UnitedStates.gif"" style=""border: 0; vertical-align: middle"" title=""UnitedStates"" width=""57""/>\n</div>",Boundary Conditions and Uncertainty Quantifications for\r\nHemodynamics,No MSC given,"<p style=""text-align: center; line-height: 2.75ex"">Advisor 1: <a href=""id.php?id=45654"">Pierre-Alain Gremaud</a><br/></p>",Dissertation: \n\nBoundary Conditions and Uncertainty Quantifications for\r\nHemodynamics,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n<a href=""http://www.ams.org/mathscinet/MRAuthorID/988642"">MathSciNet</a>\n</p>",Ph.D.,North Carolina State University,UnitedStates,2013,NaN,1,Pierre-Alain Gremaud,45654,None,0,None,0,None,0,None,0,None,0,0,Uncertainty Quantifications for\r\nHemodynamics


In [58]:
df.loc[182831,'num_descendants'] = '0'

In [59]:
df['num_descendants'] = df['num_descendants'].apply(int)

In [60]:
# recursive function to obtain set of ancestors for a record

def ancestors(rec):
    advisors = []
    for i in range(1,7):
        adv = df.loc[rec, 'Advisor_%s_MGP_ID'%(i)]
        if adv != 0:
            advisors.append(adv)
        else:
            continue
    
    if len(advisors) == 0:
        return set()
    else:
        adset = set(advisors)
        for adv in advisors:
            adset = adset.union(ancestors(adv))
        return adset
    
def anc_names(rec):
    anset = set()
    for anc in ancestors(rec):
        anset.add(df.loc[anc, 'Name'])
    print('Ancestors of ', df.loc[rec, 'Name'], 'are: ', anset, '\nNumber of ancestors is ', len(anset))
    return anset
    
    

In [61]:
def students(rec):
    stset = []
    for j in range(1,7):
        for stu in df[df['Advisor_%s_MGP_ID'%(j)] == rec].index:
            stset.append(stu)
    return set(stset)

In [62]:
def st_names(rec):
    names = set()
    for id in students(rec):
        names.add(df.loc[id,'Name'])
    print(df.loc[rec,'Name'], ' has ', len(names), ' students with names: ', names)
    return names

In [63]:
#depth first search for descendants

def desc_DFS(rec, series):
    series[rec] = True
    for v in students(rec):
        if series[v] == False:
            desc_DFS(v, series)
    return df[series]['Name']
    

In [64]:
def descendants(rec):
    D = desc_DFS(rec, pd.Series(False, index = df.index))
    return D[D.index != rec]

### 1.9. Cleaning up name column<a id='1.9._Cleaning_up_name_column'></a>

In [65]:
# remove \n from name column

df['Name'] = df['Name'].str.strip('\n')

In [66]:
df[df['Name'].str[-1] != ' ']

,Name,Degree_Info,Thesis,MSC,Advisor,Descendants,MScNID,Degree,School,Country,Year,msc,Num_advisors,Advisor_1_Name,Advisor_1_MGP_ID,Advisor_2_Name,Advisor_2_MGP_ID,Advisor_3_Name,Advisor_3_MGP_ID,Advisor_4_Name,Advisor_4_MGP_ID,Advisor_5_Name,Advisor_5_MGP_ID,Advisor_6_Name,Advisor_6_MGP_ID,num_students,num_descendants
MGP_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
193664,,"<div style=""line-height: 30px; text-align: center; margin-bottom: 1ex"">\n<span style=""margin-right: 0.5em""> <span style=""color:\n #006633; margin-left: 0.5em""></span> </span>\n</div>",,No MSC given,None,No students known.,"<p style=""text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small"">\n</p>",,,Unknown,0,NaN,0,None,0,None,0,None,0,None,0,None,0,None,0,0,0


All names have a space at the end.

In [67]:
# removing space at end

df['Name'] = df['Name'].str[:-1]

Some names have double spaces.

In [68]:
df['Name'] = df['Name'].str.replace('  ', ' ')

### 1.10. Extracting MathSciNet ID<a id='1.10._Extracting_MathSciNet_ID'></a>

In [69]:
df['MScNetID'] = df['MScNID'].apply(lambda x: None if 'href' not in x
                                                     else (x.split('ID/')[-1].split('">')[0]))

In [70]:
def none_bool(x):
    if x == None:
        return True
    else: 
        return False

In [71]:
df[df['MScNetID'].apply(none_bool)].shape

(110396, 28)

110396 rows with `'MScNetID'` value `None` because no `'href'` in the MScN string

In [72]:
df[(~df['MScNetID'].apply(none_bool)) & (df['MScNetID'].str.contains('p'))].shape

(378, 28)

In [73]:
df['MScNetID'] = df['MScNetID'].apply(lambda C: None if C == '<p style="text-align: center; margin-top: 0; margin-bottom: 0px; font-size: small'
                                      else C)


In [74]:
df[~df['MScNetID'].apply(none_bool)].shape

(148648, 28)

Check for duplicated MScNetID entries

In [75]:
mscncount = df.groupby('MScNetID').count()

dup = mscncount[mscncount['Name']>1].sort_values('Name',ascending=False)

# getting MGP_IDs
dups = list(dup.index)

df['dup_bool'] = df['MScNetID'].apply(lambda C: True if C in dups else False)

multi = df[df['dup_bool']]

multi.shape

(238, 29)

238 duplicated MScNetID entries

In [76]:
# dict of MGP_IDs:MScN_IDs that were missed due to MGP error

redo_dict = {467:483015, 
             239638:939540, 
             239639:None, 
             239640:None, 
             240390:None, 
             242852:981207, 
             240642:None,
             240643:1116130, 
             240644:None, 
             240645:None,
             242854:1340983,
             240461:1379454,
             240466:1119891,
             240476:1372483,
             193987:None,
             220405:1377934,
             220406:1299512,
             240460:None,
             240464:None,
             240470:None,
             91549:636696,
             240396:1341326,
             240398:None,
             240399:1172854,
             240400:None,
             240401:None,
             233558:None,
             233559:1326846,
             240468:None,
             240473:None,
             240462:1234379,
             240463:None,
             240392:875239,
             240393:None,
             240394:974778,
             240471:None,
             240646:1379690,
             233524:None,
             220374:None,
             239834:633323,
             45905:1254428,
             261630:None,
             240641:998355,
             239716:1380212,
             250350:None,
             203863:None,
             190248:None,
             233835:None,
             249532:1163257,
             239825:782394,
             69659:None,
             240475:None,
             106305:None,
             263857:1341609,
             234471:None,
             226090:None,
             239397:1135107,
             243561:None,
             239966:947586,
             256199:1133915,
             243588:879054,
             245235:1050614,
             236262:None,
             43447:1248472,
             6277:None,
             195084:None,
             260352:1079752,
             78303:None,
             217258:1373996,
             201634:235506,
             240027:None,
             254991:None,
             255306:882749,
             236999:None,
             216074:230367,
             60744:None,
             197423:None,
             32464:None,
             239480:None,
             244152:None,
             113745:None,
             244500:201995,
             221233:None,
             231153:None,
             248452:None,
             156801:1060570,
             201922:984063,
             233047:None,
             206025:None,
             255560:None,
             209514:None,
             207319:None,
             204339:None,
             140308:1329630,
             231230:None,
             174075:None,
             194321:None,
             229598:None,
             237224:None,
             179871:1080275,
             167227:None,
             217134:None,
             8362:None,
             227165:None,
             236542:None,
             72208:None,
             174660:None,
             230727:None,
             207645:None,
             259068:None,
             149943:None,
             255074:None,
             127674:None,
             177680:None,
             239383:None,
             204922:None,
             250218:None,
             198543:None,
             248345:None,
             240192:None,
             253506:None,
             153024:None,
             258153:None,
             227807:None,
             262337:None,
             126065:None,
             137109:None,
             194219:None,
             261293:None,
             98025:1371696,
             209193:None,
             93854:None,
             234647:916233,
             255689:None,
             106184:None,
             242196:None,
             143647:None,
             234647:None,
             5638:190631
            }

for mgpid in redo_dict.keys():
    df.loc[mgpid,'MScNetID'] = redo_dict[mgpid]
    
df['MScNetID'] = df['MScNetID'].apply(lambda C: str(C) if C!=None else C)

In [77]:
# these are the records of people with duplicate entries
# these add the information from the droped duplicate row

df.loc[242445, ['Advisor_2_Name','Advisor_2_MGP_ID']] = ['Abedelaziz Mohaisen', 167150]

df.loc[218128, ['Advisor_2_Name', 'Advisor_2_MGP_ID']] = ['Patricio Luis Felmer', 11390]

df.loc[220918,'MSC'] = 55

df.loc[233082,['num_students','num_descendants']] = [1,9]

df.loc[9633, ['Advisor_2_Name','Advisor_2_MGP_ID']] = ['Joshua Chover', 5135]

df.loc[257640,['Advisor_2_Name','Advisor_2_MGP_ID']] = ['Casimir Alexander Kulikowski', 70194]

df.loc[120115,['Advisor_4_Name','Advisor_4_MGP_ID']] = ['Remco Theodoor Peters', 120115]

df.loc[244136,['Advisor_2_Name','Advisor_2_MGP_ID']] = ['Rainer Nagel', 21589]

df.loc[256396, 'Thesis'] = 'Existence of solutions for some nonlinear elliptic problems'

df.loc[252369, ['Advisor_3_Name','Advisor_3_MGP_ID']] = ['Nikolaos D. Katopodes', 252388]

In [78]:
# These records have advisors with duplicate entries 
# and the original reference to the advisor uses the 
# entry that is being dropped.

df.loc[206047, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Nati Linial', 63895]

df.loc[238660, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Volodimir Leonidovich Makarov', 187063]

df.loc[65262, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Norbert Peters', 233082]

df.loc[223968, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Jerzy Franciszek Plebański', 119928]

df.loc[197270, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Jorge Manuel Sotomayor Tello', 69192]

df.loc[199124, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['David José Fernández Cabrera', 223970]

df.loc[238114, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Xiang-Gen Xia', 173753]

df.loc[242961, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Gui-Qiang G. Chen', 38667]

df.loc[178268, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Hongze Li (李红泽)', 204934]

df.loc[207215, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Marco Maria Luigi Sammartino', 188663]

df.loc[232091, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Hector Fidencio Sanchez-Morgado', 6852]

df.loc[231388, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Adriano Tomassini', 171044]

df.loc[45308, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['David John Stevenson', 227187]

df.loc[215102, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['K. Suresh Kumar', 123774]

df.loc[212856, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Frederic Paik Schoenberg', 34439]

df.loc[243237, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Vitor Domingos Martins de Araújo', 54150]

df.loc[243238, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Vitor Domingos Martins de Araújo', 54150]

df.loc[257136, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Zhi Ding', 220353]

df.loc[257137, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Zhi Ding', 220353]

df.loc[237126, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Jayanthan A V', 37588]

df.loc[241593, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Mrinal Kanti Das', 9777]

df.loc[206697, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Abbas Syed', 141151]

df.loc[239385, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Dimas Jose Goncalves', 146882]

df.loc[250923, ['Advisor_1_Name', 'Advisor_1_MGP_ID']] = ['Sourav Pal', 251547]

df.loc[239940, ['Advisor_2_Name', 'Advisor_2_MGP_ID']] = ['José Ángel Peláez', 77913]

df.loc[258001, ['Advisor_2_Name', 'Advisor_2_MGP_ID']] = ['Ragnar Freij-Hollanti', 165053]



Check for any other `MScNetID` duplicates

In [79]:
mscncount = df.groupby('MScNetID').count()

dup = mscncount[mscncount['Name']>1].sort_values('Name',ascending=False)

# getting MGP_IDs
dups = list(dup.index)

df['dup_bool'] = df['MScNetID'].apply(lambda C: True if C in dups else False)

multi = df[df['dup_bool']]

multi.shape

(0, 29)

In [80]:
df[~df['MScNetID'].apply(none_bool)].shape

(148554, 29)

In [81]:
# redo_list = [i for i in redo_dict.values() if i!= None]
# len(redo_list)

In [82]:
# pd.Series(redo_list).to_csv('MScN/MScN_redo_list.csv')

In [83]:
df.groupby('MScNetID').count().shape

(148554, 28)

### 1.11. More drops<a id='1.11._More_drops'></a>

In [84]:
df.drop(['MSC','Advisor','Descendants','MScNID','Degree_Info','dup_bool'], axis = 1, inplace = True)

# change some column names
df.columns = ['Name', 'Thesis', 'Degree', 'School', 'Country', 'Year', 'MSC', 'num_advisors', 'Advisor_1_Name',
       'Advisor_1_MGP_ID', 'Advisor_2_Name', 'Advisor_2_MGP_ID',
       'Advisor_3_Name', 'Advisor_3_MGP_ID', 'Advisor_4_Name',
       'Advisor_4_MGP_ID', 'Advisor_5_Name', 'Advisor_5_MGP_ID',
       'Advisor_6_Name', 'Advisor_6_MGP_ID', 'num_students',
       'num_descendants', 'MScNetID']

# change order of columns

df = df[['Name', 'MScNetID', 'Degree', 'School', 'Country', 'Year', 'Thesis', 'MSC', 'num_students',
       'num_descendants', 'num_advisors', 'Advisor_1_Name',
       'Advisor_1_MGP_ID', 'Advisor_2_Name', 'Advisor_2_MGP_ID',
       'Advisor_3_Name', 'Advisor_3_MGP_ID', 'Advisor_4_Name',
       'Advisor_4_MGP_ID', 'Advisor_5_Name', 'Advisor_5_MGP_ID',
       'Advisor_6_Name', 'Advisor_6_MGP_ID']]

cols = ['Name', 'MScNetID', 'Degree', 'School', 'Country', 'Year', 'Thesis', 'MSC', 'num_students',
       'num_descendants', 'num_advisors', 'Advisor_1_Name',
       'Advisor_1_MGP_ID', 'Advisor_2_Name', 'Advisor_2_MGP_ID',
       'Advisor_3_Name', 'Advisor_3_MGP_ID', 'Advisor_4_Name',
       'Advisor_4_MGP_ID', 'Advisor_5_Name', 'Advisor_5_MGP_ID',
       'Advisor_6_Name', 'Advisor_6_MGP_ID']

In [85]:
df.shape

(259422, 23)

## 2. MScN data frame<a id='2._MScN_data_frame'></a>

### 2.1. Import MScN data<a id='2.1._Import_MScN_data'></a>

In [86]:
#  ms_list = [pd.read_csv('MScN/MScN_Author%s,%s.csv'%(K*1000, (K+1)*1000)) for K in range(0,150)]+[pd.read_csv('MScN/MScN_Author0,44.csv')]+[pd.read_csv('MScN/MScN_Author0,387.csv')]

#  ms = pd.concat(ms_list,ignore_index=True)

# ms.to_csv('MScN/MScN_Author_scrape.csv')

In [87]:
ms = pd.read_csv('MScN/MScN_Author_scrape.csv')

In [88]:
ms.shape

(148796, 6)

### 2.2. Drop defective rows<a id='2.2._Drop_defective_rows'></a>

In [89]:
ms['MScN_ID'] = ms['MScN_ID'].apply(str)

In [90]:
# drop non-digit MScN_ID entries


ms = ms[ms['MScN_ID'].str.isdigit()]

In [91]:
# drop duplicates in ms table
ms.drop_duplicates(subset = 'MScN_ID', inplace=True)

In [92]:
ms.shape

(148191, 6)

In [93]:
mscount = ms.groupby('MScN_ID').count().sort_values('Name',ascending=False)

msdup = mscount[mscount['Name']>1]

# getting MGP_IDs
msdups = list(msdup.index)

ms['msdup_bool'] = ms['MScN_ID'].apply(lambda C: True if C in msdups else False)

msmulti = ms[ms['msdup_bool']]

msmulti.shape

(0, 7)

In [94]:
# drop MScN_ID because it doesn't correspond to the proper person

ms = ms[~(ms['MScN_ID'] == '713534')]

### 2.3. Align MGP and MScN MathSciNet IDs<a id='2.3._Align_MGP_and_MScN_MathSciNet_IDs'></a>

In [95]:
ms_mscns = set(ms['MScN_ID'])
df_mscns = set(df[~df['MScNetID'].apply(none_bool)]['MScNetID'])

print(len(ms_mscns))
print(len(df_mscns))
print(len(df_mscns) - len(ms_mscns))

148190
148554
364


In [96]:
in_ms = ms_mscns.difference(df_mscns)
print(len(in_ms))
print(in_ms)

0
set()


In [97]:
# pd.Series(list(in_ms)).to_csv('missed_mscns2.csv')

In [98]:
in_df = df_mscns.difference(ms_mscns)
len(in_df)

364

The rest of the mscn_ids in `df` that are not in `ms` do not correspond to an actual MathSciNet page.  Some of these can maybe be fixed, the set is small enough to leave.

In [99]:
for x in list(in_df):
    df['MScNetID'] = df['MScNetID'].apply(lambda C: None if C == x else C)


Checking that the MathSciNet IDs align

In [100]:
ms_mscns = set(ms['MScN_ID'])
df_mscns = set(df[~df['MScNetID'].apply(none_bool)]['MScNetID'])

print(len(ms_mscns))
print(len(df_mscns))
print(len(df_mscns) - len(ms_mscns))

148190
148190
0


In [101]:
in_ms = ms_mscns.difference(df_mscns)
print(len(in_ms))


0


In [102]:
in_df = df_mscns.difference(ms_mscns)
len(in_df)

0

In [103]:
intersect = df_mscns.intersection(ms_mscns)
len(intersect)

148190

### 2.4. Extracting info from scraped MathSciNet data<a id='2.4._Extracting_info_from_scraped_MathSciNet_data'></a>

In [104]:
ms.shape

(148190, 7)

In [105]:
ms.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'MScN_ID', 'Name', 'Data', 'Collab_Subj',
       'msdup_bool'],
      dtype='object')

In [106]:
ms.set_index('MScN_ID', inplace=True)

ms.drop(['Unnamed: 0', 'Unnamed: 0.1', 'msdup_bool'], axis=1, inplace=True)

# getting Name
ms['Name1'] = ms['Name'].apply(lambda C: C.split('\n    ')[-1].split(' - ')[0])


# getting earliest indexed publication
ms['Earliest_Pub'] = ms['Data'].apply(lambda C: C.split('<strong>')[-1].split('</strong')[0])

# converting missing earliest pub data to 0
ms['Earliest_Pub'] = ms['Earliest_Pub'].apply(lambda C: C if C.isdigit() else '0')

# convert earliest pub to type int
ms['Earliest_Pub'] = ms['Earliest_Pub'].apply(int)


# getting total pubs
ms['Total_Pubs'] = ms['Data'].apply(lambda C: C.split('Total Publications:</td>, <td>')[-1].split('<')[0])

# replace missing total pub values with 0
ms['Total_Pubs'] = ms['Total_Pubs'].apply(lambda C: '0' if C == '[' else C)

# convert total pub column to type int
ms['Total_Pubs'] = ms['Total_Pubs'].apply(int)


# getting total related publications
ms['Total_Rel_Pubs'] = ms['Data'].apply(lambda C: C.split('Total Related Publications:</td>, <td>')[-1].split('<')[0] 
                                        if 'Related' in C
                                        else '0')

# convert total rel pubs column to type int
ms['Total_Rel_Pubs'] = ms['Total_Rel_Pubs'].apply(int)


# getting total Citations
ms['Total_Citations'] = ms['Data'].apply(lambda C: C.split('Total Citations:</td>, <td>')[-1].split('<')[0])

# replace missing citation values with 0
ms['Total_Citations'] = ms['Total_Citations'].apply(lambda C: '0' if C == '[' else C)

# convert citation column to type int
ms['Total_Citations'] = ms['Total_Citations'].apply(int)

def num_collabs(s):
    return len(s.split('MR Author ID ')) - 1

def col_list(s):
    
    return [[s.split('MR Author ID ')[i+1].split('">')[0], 
             s.split('MR Author ID ')[i+1].split('">')[1].split('</a>')[0]]
           for i in range(num_collabs(s))]



def col_ids(ls):
    return [x[0] for x in ls]
def col_names(ls):
    return [x[1] for x in ls]

# getting tuple of collaborator ids and names
ms['Collaborator_Data'] = ms['Collab_Subj'].apply(col_list)

# getting list of collaborator ids
ms['Collaborator_IDs'] = ms['Collaborator_Data'].apply(col_ids)

# getting list of collaborator names
ms['Collaborator_Names'] = ms['Collaborator_Data'].apply(col_names)

def num_subjects(s):
    return len(s.split('Primary Class ')) - 1

def subject_list(s):
    
    return [s.split('Primary Class ')[i+1].split('">')[0]
           for i in range(num_subjects(s))]

# getting list of subjects (in MSC)
ms['Subjects'] = ms['Collab_Subj'].apply(subject_list)

ms.drop(['Name', 'Data', 'Collab_Subj'], axis = 1, inplace = True)

ms.columns = ['Name_MScN', 'Earliest_Pub', 'Total_Pubs', 'Total_Rel_Pubs', 'Total_Citations', 'CollabData', 'CollabIDs', 'CollabNames', 'Subjects']

ms.drop('CollabData', axis=1, inplace=True)

ms['Num_Collaborators'] = ms['CollabIDs'].apply(len)

ms['Num_Subjects'] = ms['Subjects'].apply(len)

ms['Total_Citations'] = ms['Total_Citations'].apply(lambda C: '0' if C == '[' else C)



ms['CollabIDs'] = ms['CollabIDs'].apply(lambda C: None if C == [] else C)
ms['CollabNames'] = ms['CollabNames'].apply(lambda C: None if C == [] else C)

ms['Subjects'] = ms['Subjects'].apply(lambda C: None if C == [] else C)



### 2.5. Save dataframes<a id='2.5._Save_dataframes'></a>

In [107]:
df.to_csv('MGP/MGP_clean.csv')
ms.to_csv('MScN/MScN_Author_clean.csv')

## 3. Joining MGP and MScN<a id='3._Joining_MGP_and_MScN'>

### 3.1. Join<a id='3.1._Join'></a>

In [108]:
am = df.join(ms, how='left', on = 'MScNetID', rsuffix = 'MScN')

In [109]:
am.shape

(259422, 33)

### 3.2. Typecast<a id='3.2._Typecast'></a>

In [110]:
# convert MScNetID, MSC, earliest pub, total pubs, total rel pubs, total citations, num collaborators, and num_subjects 
# to type int with -1 serving for empty info

cols = ['Earliest_Pub', 'Total_Pubs', 'Total_Rel_Pubs', 'Total_Citations', 'Num_Collaborators', 'Num_Subjects','MSC', ]

for col in cols:
    am[col] = am[col].apply(str)\
    .apply(lambda C: -1 if C == 'nan' else int(float(C)))

am['MScNetID'] = am['MScNetID'].apply(str).apply(lambda C: -1 if C == 'None' else int(C))


In [111]:
list(zip(am.columns, [am[c].dtype for c in am.columns]))


[('Name', dtype('O')),
 ('MScNetID', dtype('int64')),
 ('Degree', dtype('O')),
 ('School', dtype('O')),
 ('Country', dtype('O')),
 ('Year', dtype('int64')),
 ('Thesis', dtype('O')),
 ('MSC', dtype('int64')),
 ('num_students', dtype('int64')),
 ('num_descendants', dtype('int64')),
 ('num_advisors', dtype('int64')),
 ('Advisor_1_Name', dtype('O')),
 ('Advisor_1_MGP_ID', dtype('int64')),
 ('Advisor_2_Name', dtype('O')),
 ('Advisor_2_MGP_ID', dtype('int64')),
 ('Advisor_3_Name', dtype('O')),
 ('Advisor_3_MGP_ID', dtype('int64')),
 ('Advisor_4_Name', dtype('O')),
 ('Advisor_4_MGP_ID', dtype('int64')),
 ('Advisor_5_Name', dtype('O')),
 ('Advisor_5_MGP_ID', dtype('int64')),
 ('Advisor_6_Name', dtype('O')),
 ('Advisor_6_MGP_ID', dtype('int64')),
 ('Name_MScN', dtype('O')),
 ('Earliest_Pub', dtype('int64')),
 ('Total_Pubs', dtype('int64')),
 ('Total_Rel_Pubs', dtype('int64')),
 ('Total_Citations', dtype('int64')),
 ('CollabIDs', dtype('O')),
 ('CollabNames', dtype('O')),
 ('Subjects', dtype('O'

In [112]:
am[(am['Total_Pubs']!= -1)].shape

(148190, 33)

### 3.3. Viable rows<a id='3.3._Viable_rows'></a>

In [113]:
am[(am['num_students'] > 0) & (am['Total_Pubs']!= -1)].shape

(41037, 33)

41,037 records with nonempty publication data and with at least one student.  This is an excellent size to use for training/testing.

In [114]:
am.to_csv('MGP_MScN.csv')
am.to_excel('MGP_MScN.xlsx')